# Cryosphere model Comparison tool (CmCt) --- IMBIE

The CmCt IMBIE tool compares user-uploaded ice sheet modeled mass change to reconciled mass change observations from the Ice-sheet Mass Balance Intercomparison Exercise (IMBIE). The IMBIE data is provided as ice-sheet-integrated time series of mass change. In the future, IMBIE will partition mass change into drainage basins, as well as into total mass balance, surface mass balance, and dynamic mass balance and this tool is designed to be able to process those. The CmCt uses a drainage basin mask to partition modeled mass change into the separate basins and sums mass changes across all basins (`Masked_Total` in the output results). The CmCt also sums modeled mass change for the entire gridded model, without applying any basin masking ('Unmasked_Total` in the output results). Note that these two sums may be different, if the gridded model file contains grid cells that are outside of the IMBIE drainage basin mask.

## Input data requirements

The input ice sheet model needs to be provided as a netCDF file. The user may upload a single input file that includes multiple years; future enhancements to this tool will allow users to upload model ensembles.

There are several requirements for the comparison:

### `Lithk` variable

The CmCt Grace Mascon tool expects the uploaded model to contain thickness data (the `lithk` variable) for the comparison.

### Rectangular grid

At time of writing, models *must* be defined on a rectangular X-Y grid in the ISMIP6 standard projected polar-stereographic space. (Note, NOT a lat-lon grid!) The ISMIP6 standard projection is defined [here](https://theghub.org/groups/ismip6/wiki). 

### Date range

The gravimetry data spans 04/2002 to 12/2023. The user can select start and end dates within this span as part of the setup for the tool.

In [1]:
import os, sys
import cftime 

# Add the directory containing 'cmct' to the Python path
# Navigate two levels up to reach main CmCt dir
cmct_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir, os.pardir))
# Import utilities for this comparison
sys.path.insert(0,cmct_dir)
from cmct.imbie import *

# Suppress numpy.dtype size changed warnings
import warnings
warnings.filterwarnings('ignore')


### Configure IMBIE comparison

In [2]:
# Flag for the ice sheet region Greenland or Antarctica
icesheet = 'GIS' # Select 'AIS' or 'GIS'

# Start and end dates for comparison
start_date = '2007-01-01'
end_date ='2009-01-01'

# Density of ice used in the model
rho_ice = 918 # (kg/m^3)

# Output file directory
output_path = cmct_dir+ '/notebooks/IMBIE/'

# Select IMBIE variable for mass change comparision (name of column in IMBIE dataset
mass_balance_column = 'Cumulative mass balance (Gt)'
if mass_balance_column == 'Cumulative mass balance (Gt)':
    mass_balance_type = 'total'
elif mass_balance_column == 'Cumulative dynamics mass balance anomaly (Gt)':
    mass_balance_type = 'dynamic'

# Set model model filename, shapefile for basin partitioning, projection, and IMBIE dataset filename
if icesheet == 'GIS':
    # Projection
    projection = 'EPSG:3413'
    
    # Model filename
    nc_filename = cmct_dir + '/test/gris_dmdt_filt.nc'
    
    # Shapefile for basin partitioning
    shape_filename = cmct_dir + '/data/IMBIE/Greenland_Basins_PS_v1.4.2/Greenland_Basins_PS_v1.4.2.shp'
    
    # IMBIE dataset filename
    obs_filename = cmct_dir + '/data/IMBIE/imbie_greenland_2021_Gt.csv'

    # Other regions --- these are used for Antarctica only, so set them to None for Greenland
    obs_east_filename = None
    obs_west_filename = None
    obs_peninsula_filename = None
    
elif icesheet == 'AIS':
    # Projection
    projection = 'EPSG:3031'
    
    # Model filename
    nc_filename = cmct_dir + '/test/ais_dmdt_grounded_filt.nc'
    
    # Shapefile for basin partitioning
    shape_filename = cmct_dir + '/data/IMBIE/ANT_Basins_IMBIE2_v1.6/ANT_Basins_IMBIE2_v1.6.shp'
    
    # IMBIE dataset filename
    obs_filename = cmct_dir + '/data/IMBIE/imbie_antarctica_2021_Gt.csv'
    
    # Other regions --- these are used for Antarctica and are not yet published, so set them to None for now
    obs_east_filename = None
    obs_west_filename = None
    obs_peninsula_filename= None

else:
    raise ValueError('Invalid icesheet value. Must be \'Greenland\' or \'Antarctica\'.')
    

In [3]:
# Check if observation file exists
if not os.path.exists(obs_filename):
    raise FileNotFoundError(f"Observation file not found: {obs_filename}")

# Check if model file exists    
if not os.path.exists(nc_filename):
    raise FileNotFoundError(f"Model file not found: {nc_filename}")

if icesheet == 'AIS':
    if (obs_east_filename and os.path.exists(obs_east_filename)) and \
           (obs_west_filename and os.path.exists(obs_west_filename)) and \
           (obs_peninsula_filename and os.path.exists(obs_peninsula_filename)):
        # Check if regional observation files exist 
        if not os.path.exists(obs_east_filename):
            raise FileNotFoundError(f"Observation file not found: {obs_east_filename}")
        if not os.path.exists(obs_west_filename):
            raise FileNotFoundError(f"Observation file not found: {obs_west_filename}")
        if not os.path.exists(obs_peninsula_filename):
            raise FileNotFoundError(f"Observation file not found: {obs_peninsula_filename}")

## Mass change comparision processing

In [4]:
# Open model file
mod_ds = xr.open_dataset(nc_filename,use_cftime=True)
time_var = mod_ds['time']

# Convert start/end comparison times to fractional year
calendar_type = time_var.to_index().calendar
start_date_dt = datetime.datetime.strptime(start_date, '%Y-%m-%d')
end_date_dt = datetime.datetime.strptime(end_date, '%Y-%m-%d')

# Adjust day to be 30 ( to avoid error if it's the 31st day in a 360_day calendar)
start_date_cftime = cftime.datetime(start_date_dt.year, start_date_dt.month, min(start_date_dt.day, 30), calendar=calendar_type)
end_date_cftime = cftime.datetime(end_date_dt.year, end_date_dt.month, min(end_date_dt.day, 30), calendar=calendar_type)

start_date_fract = start_date_cftime.year + (start_date_cftime.dayofyr-1) / days_in_year(start_date_cftime)
end_date_fract = end_date_cftime.year + (end_date_cftime.dayofyr-1) / days_in_year(end_date_cftime)


In [5]:
# Read IMBIE mass change from start to end date
IMBIE_mass_change = process_imbie_data(obs_filename, start_date_fract, end_date_fract, mass_balance_column)

# Calculate time-varying modeled mass change
model_mass_change = process_model_data(mod_ds,time_var, IMBIE_mass_change, start_date_cftime, end_date_cftime, start_date_fract, end_date_fract, rho_ice, projection, shape_filename, icesheet)

# Calculate IMBIE-model mass change residuals
imbie_model_residuals = calculate_model_imbie_residuals(start_date_fract, end_date_fract, icesheet, model_mass_change, IMBIE_mass_change, mass_balance_column, obs_east_filename, obs_west_filename, obs_peninsula_filename)


The selected dates {start_date_cftime} and {end_date_cftime} are within the range of the model data.


## Display results and write output csv file

In [6]:
# Extract the base name of the nc file (without .nc extension)
nc_base_filename = os.path.basename(nc_filename).replace('.nc', '')

# Create the CSV filename by combining the output path and the base nc filename with .csv extension
csv_filename = os.path.join(output_path, f"{nc_base_filename}.csv")

# Write output csv file and display results
write_and_display_mass_change_comparison_all_dates(icesheet, model_mass_change, imbie_model_residuals, mass_balance_type, start_date_fract, end_date_fract, csv_filename)
   


 Time-varying Mass change comparison (total): 2007.0 - 2009.0
Date            Basin/Region         Model mass change (Gt)    IMBIE mass change (Gt)    Residual (Gt)       
2007.0          Masked_Total         0.0                       0.0                       0.0                 
2007.0833       Masked_Total         -16.42                    -19.57                    -3.15               
2007.1667       Masked_Total         -32.86                    -39.46                    -6.60               
2007.25         Masked_Total         -49.28                    -59.75                    -10.47              
2007.3333       Masked_Total         -65.70                    -80.36                    -14.66              
2007.4167       Masked_Total         -82.15                    -101.17                   -19.03              
2007.5          Masked_Total         -98.57                    -121.99                   -23.42              
2007.5833       Masked_Total         -114.99             